# Probability Utility 的設定輸出

In [1]:
from model_EndoU import *
from OptT3_EndoU import *

In [2]:
def optimal_output(m=0.2,mask_supply="benchmark",protect_set="benchmark",S0=0.99,𝛾=1-(17/18)**14, α=0.0138):
    
    if mask_supply=="benchmark":
        m0, m1, m2= m,m,m
    elif mask_supply=="growth":
        m0, m1, m2= m,m,1.15*m
    elif mask_supply=="init_stock":
        m0, m1, m2= 1.15*m,m,m
        
    if protect_set=="benchmark":
        σn, δn = 0.7, 0.7
        σo, δo = 0.5, 0.5
    elif protect_set=="lower_inward":
        σn, δn = 0.6, 0.7
        σo, δo = 0.4, 0.5
    elif protect_set=="similar_prot":
        σn, δn = 0.7, 0.7
        σo, δo = 0.69, 0.69
    
    optimal_model = nation_SIRD(S0=S0,σn=σn, σo=σo, δn=δn, δo=δo, 𝛾=𝛾, α=α)
    obj,πA,πB,πA1,πB1,πA0,πB0 = optimal_model.find_optimal(m0=m0,m1=m1,m2=m2)
    
    return obj

def output(mask_supply="benchmark",protect_set="benchmark",I=0.01,𝛾=1-(17/18)**14,𝛼=0.0138):
    
    m_vals=np.linspace(0.1,0.8,71)
    
    GRBT=map(lambda m: solve_GRBT(m,mask_supply,protect_set,I,𝛾,𝛼),m_vals)
    GRBTdata= pd.DataFrame.from_records(list(GRBT), columns =['m',
                                                              'share of sign up, without mask',
                                                              'share of sign up, with mask',
                                                              'ICphi','ICn',
                                                              'πB','πA','πA0','πB0','πA1','πB1',
                                                              'P2ϕ','P3ϕ','GRBT fatalities'])
    SRA1=map(lambda m: solve_SRA1(m,mask_supply,protect_set,I,𝛾,𝛼),m_vals)
    SRA1data= pd.DataFrame.from_records(list(SRA1), columns =['m','πB','πA','πA0','πB0','πA1','πB1','SRA1 fatalities'])

    SRA2=map(lambda m: solve_SRA2(m,mask_supply,protect_set,I,𝛾,𝛼),m_vals)
    SRA2data= pd.DataFrame.from_records(list(SRA2), columns =['m','πB','πA','πA0','πB0','πA1','πB1','SRA2 fatalities'])
    
    Optimal=map(lambda m: optimal_output(m,mask_supply,protect_set,1-I,𝛾,𝛼),m_vals)
    Optldata= pd.Series(list(Optimal), name ='Optimal fatalities')
    
    # combined death levels and ICs of GRBT into new dataframe
    df_combined = pd.concat([GRBTdata['m'],GRBTdata['ICphi'],GRBTdata['ICn'],
                             GRBTdata['P2ϕ'],GRBTdata['P3ϕ'],
                             GRBTdata['GRBT fatalities'],SRA1data['SRA1 fatalities'],
                             SRA2data['SRA2 fatalities'],Optldata], axis=1)
    df_combined = df_combined.rename(columns={'P2ϕ':'P2phi','P3ϕ':'P3phi'})
    
    return df_combined

In [3]:
graph1=output()
graph1.to_csv('output/230504 EndoU benchmark, graph.csv')

In [4]:
graph2=output(mask_supply="growth",protect_set="benchmark")
graph2.to_csv('output/230504 EndoU growth, graph.csv')

In [5]:
graph3=output(mask_supply="init_stock",protect_set="benchmark")
graph3.to_csv('output/230504 EndoU init_stock, graph.csv')

In [6]:
graph4=output(mask_supply="benchmark",protect_set="lower_inward")
graph4.to_csv('output/230504 EndoU lower_inward, graph.csv')

In [7]:
graph5=output(mask_supply="benchmark",protect_set="similar_prot")
graph5.to_csv('output/230504 EndoU similar_prot, graph.csv')

In [8]:
graph6=output(mask_supply="benchmark",protect_set="benchmark",𝛾=0.9)
graph6.to_csv('output/230504 EndoU gamma09, graph.csv')

In [9]:
graph7=output(mask_supply="benchmark",protect_set="benchmark",I=0.3,α=1)
graph7.to_csv('output/230504 EndoU extreme, graph.csv')

In [10]:
graph8=output(mask_supply="benchmark",protect_set="benchmark",α=1)
graph8.to_csv('output/230504 EndoU alpha1, graph.csv')

In [11]:
graph9=output(mask_supply="benchmark",protect_set="benchmark",I=0.3)
graph9.to_csv('output/230504 EndoU I03, graph.csv')

# Benchmark Utility設定

In [1]:
from model import *
from OptT3_benchmark import *

In [2]:
def optimal_output(m=0.2,utility_set="benchmark",mask_supply="benchmark",protect_set="benchmark",
                   S0=0.99,𝛾=1-(17/18)**14, α=0.0138,𝛽=2.4/(18/14)):
    
    if mask_supply=="benchmark":
        m0, m1, m2= m,m,m
    elif mask_supply=="growth":
        m0, m1, m2= m,m,1.15*m
    elif mask_supply=="init_stock":
        m0, m1, m2= 1.15*m,m,m
        
    if protect_set=="benchmark":
        σn, δn = 0.7, 0.7
        σo, δo = 0.5, 0.5
    elif protect_set=="lower_inward":
        σn, δn = 0.6, 0.7
        σo, δo = 0.4, 0.5
    elif protect_set=="similar_prot":
        σn, δn = 0.7, 0.7
        σo, δo = 0.69, 0.69
    elif protect_set=="zombie":
        σn, δn = 0.7, 1
        σo, δo = 0.5, 1
    elif protect_set=="example":
        σn, δn = 1, 1
        σo, δo = 1, 1
        
    if utility_set=="benchmark":
        vn, vo, 𝜌 = 0.7, 0.5, 1
    elif utility_set=="denp_ut":
        vn, vo, 𝜌 = σn, σo, 1
    elif utility_set=="lower_disc":
        vn, vo, 𝜌 = 0.7, 0.5, 0.5
    
    optimal_model = nation_SIRD(S0=S0,σn=σn, σo=σo, δn=δn, δo=δo, 𝛽=𝛽, 𝛾=𝛾, v=vo/vn, ρ=ρ, α=α)
    obj,πA,πB,πA1,πB1,πA0,πB0 = optimal_model.find_optimal(m0=m0,m1=m1,m2=m2)
    
    return obj,πA,πB,πA1,πB1,πA0,πB0

def output(utility_set="benchmark",mask_supply="benchmark",protect_set="benchmark",
           I=0.01,𝛾=1-(17/18)**14,𝛼=0.0138,𝛽=2.4/(18/14)):
    
    m_vals=np.linspace(0.1,0.8,71)
    
    GRBT=map(lambda m: solve_GRBT(m,utility_set,mask_supply,protect_set,I,𝛾,𝛼,𝛽),m_vals)
    GRBTdata= pd.DataFrame.from_records(list(GRBT), columns =['m',
                                                              'share of sign up, without mask',
                                                              'share of sign up, with mask',
                                                              'ICphi','ICn',
                                                              'πB','πA','πA0','πB0','πA1','πB1',
                                                              'GRBT fatalities','Uphi','Un','Udiff','P2ϕ','P3ϕ'])
    SRA1=map(lambda m: solve_SRA1(m,utility_set,mask_supply,protect_set,I,𝛾,𝛼,𝛽),m_vals)
    SRA1data= pd.DataFrame.from_records(list(SRA1), columns =['m','πB','πA','πA0','πB0','πA1','πB1',
                                                              'SRA1 fatalities','Uphi','Un','Udiff'])

    SRA2=map(lambda m: solve_SRA2(m,utility_set,mask_supply,protect_set,I,𝛾,𝛼,𝛽),m_vals)
    SRA2data= pd.DataFrame.from_records(list(SRA2), columns =['m','πB','πA','πA0','πB0','πA1','πB1',
                                                              'SRA2 fatalities','Uphi','Un','Udiff'])
    
    Optimal=map(lambda m: optimal_output(m,utility_set,mask_supply,protect_set,1-I,𝛾,𝛼,𝛽),m_vals)
    Optldata=pd.DataFrame.from_records(list(Optimal), columns =['Optimal fatalities','πA','πB','πA1','πB1','πA0','πB0'])

    df_combined = pd.concat([GRBTdata['m'],GRBTdata['ICphi'],GRBTdata['ICn'],GRBTdata['P2ϕ'],GRBTdata['P3ϕ'],
                             GRBTdata['GRBT fatalities'],SRA1data['SRA1 fatalities'],
                             SRA2data['SRA2 fatalities'],Optldata['Optimal fatalities']], axis=1)
    df_combined = df_combined.rename(columns={'P2ϕ':'P2phi','P3ϕ':'P3phi'})
    
    return df_combined

In [3]:
g1=output(utility_set="benchmark",mask_supply="benchmark",protect_set="benchmark",I=0.3,α=1)
g1.to_csv('output/230428 benchmarkU extreme, graph.csv')

D:\google 雲端硬碟\筆電\code\efficient facemask distribution\model.py:105: RuntimeWarning: divide by zero encountered in double_scalars
  πB = min(m1/πB_coef,1)
D:\google 雲端硬碟\筆電\code\efficient facemask distribution\model.py:114: RuntimeWarning: divide by zero encountered in double_scalars
  πA0 = min(m2/π2_0_coef,1)


In [4]:
graph1=output(utility_set="benchmark",mask_supply="benchmark",protect_set="benchmark",I=0.01,α=1)
graph1.to_csv('output/230428 benchmarkU alpha1, graph.csv')

D:\google 雲端硬碟\筆電\code\efficient facemask distribution\model.py:102: RuntimeWarning: divide by zero encountered in double_scalars
  πB = min(m1/πB_coef,1)
D:\google 雲端硬碟\筆電\code\efficient facemask distribution\model.py:111: RuntimeWarning: divide by zero encountered in double_scalars
  πA0 = min(m2/π2_0_coef,1)


In [5]:
graph1=output(utility_set="benchmark",mask_supply="benchmark",protect_set="benchmark",I=0.3,α=0.0148)
graph1.to_csv('output/230428 benchmarkU I03, graph.csv')

D:\google 雲端硬碟\筆電\code\efficient facemask distribution\model.py:102: RuntimeWarning: divide by zero encountered in double_scalars
  πB = min(m1/πB_coef,1)
D:\google 雲端硬碟\筆電\code\efficient facemask distribution\model.py:111: RuntimeWarning: divide by zero encountered in double_scalars
  πA0 = min(m2/π2_0_coef,1)


In [3]:
graph1=output(utility_set="benchmark",mask_supply="benchmark",protect_set="example",𝛾=1,𝛼=1,𝛽=1)
graph1.to_csv('output/230509 benchmarkU all1, graph.csv')

D:\google 雲端硬碟\筆電\code\efficient facemask distribution\model.py:105: RuntimeWarning: divide by zero encountered in double_scalars
  πB = min(m1/πB_coef,1)
D:\google 雲端硬碟\筆電\code\efficient facemask distribution\model.py:114: RuntimeWarning: divide by zero encountered in double_scalars
  πA0 = min(m2/π2_0_coef,1)
